In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.insert(0,'../')

import requests
from tqdm.notebook import tqdm

In [2]:
import pymongo
import pprint
import json
from src.mongo_utils import collection
from src.mlflow_utils import mlflow
from src.ha_utils import set_sensor_state, Input


api_endpoint = f"http://127.0.0.1:8000/mixtral/"
api_endpoint = f"http://127.0.0.1:8080/generate/"

In [3]:
from src.mongo_utils import get_database_entry, create_set


with open('../test_set1.json') as file:
    test_set_data = json.load(file)
test_set = [a for subl in test_set_data.values() for a in subl]

In [7]:
dataset = []
for data in test_set:
    dataset += create_set(get_database_entry(data))

In [3]:
with open('../test_set1_postprocessed.json', 'r') as file:
    dataset = json.load(file)

In [7]:
import pyarrow as pa
import pyarrow.parquet as pq
import os
import pandas as pd

dataset

def write_split(split, base_path="../dataset/v0/", overwrite=False):
    if not os.path.exists(base_path):
        os.mkdir(base_path)
    path = os.path.join(base_path, f"{split}.parquet")
    if os.path.exists(path):
        if overwrite:
            os.remove(path)
        else:
            raise Exception("Already existing!")
        data.append(value)
    df = pd.DataFrame(data=dataset, columns=["title", "context_word", "context_sentence", "gt"])
    table = pa.Table.from_pandas(df)
    pq.write_table(table, path)

write_split("train")

In [4]:
import random
random.seed(42)
random.shuffle(dataset)

In [4]:
import pandas as pd

eval_data = {
    "inputs": [],
    "ground_truth": [],
    "prompt_data": []
}
for title, context_word, context_sentence, gt in dataset:
    test = {}
    test['title'] = title
    test['context_word'] = context_word
    test['context_sentence'] = context_sentence.replace("'", "")
    eval_data['inputs'] += [f"{context_word} --- {context_sentence}"]
    eval_data['prompt_data'] += [test]
    eval_data['ground_truth'] += [gt]
eval_data = pd.DataFrame(eval_data)
eval_data

,inputs,ground_truth,prompt_data
0,Liebe --- Die ''Liebe'' überwindet alle Grenzen.,inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
1,Liebe --- „Die ''Liebe'' ist in der Literatur ...,inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
2,Liebe --- „Im Mittelpunkt stand die ''Liebe''.“,inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
3,"Liebe --- „Die Gefühle, die sie durchströmen u...",inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
4,"Liebe --- „Erst zu Beginn der Fastenzeit, nach...",inniges Gefühl der Zuneigung für jemanden oder...,"{'title': 'Liebe', 'context_word': 'Liebe', 'c..."
...,...,...,...
1102,"Acht --- „Jetzt verhängt, wie Dr. Robert mitte...",Ausschluss aus einer Gemeinschaft; Verurteilun...,"{'title': 'Acht', 'context_word': 'Acht', 'con..."
1103,Acht --- „Seither tickt im ‚Runden Eck‘ eine Z...,Ausschluss aus einer Gemeinschaft; Verurteilun...,"{'title': 'Acht', 'context_word': 'Acht', 'con..."
1104,"Acht --- „Der Kleine widerſtrebte, ſteif und u...","Aufmerksamkeit, Beachtung; Fürsorge","{'title': 'Acht', 'context_word': 'Acht', 'con..."
1105,Acht --- „Worüber ist das Buch: sagt Marie end...,"Aufmerksamkeit, Beachtung; Fürsorge","{'title': 'Acht', 'context_word': 'Acht', 'con..."


In [20]:
import ipywidgets as widgets
out = widgets.Output(layout={'border': '1px solid black'})

In [7]:
import asyncio
import aiohttp

async def async_prompting(index, prompt, session, entrypoint="http://127.0.0.1:8080/generate"):
    try:
        json_body = {"inputs": prompt,
                     "parameters": {"temperature": 0.001, "max_new_tokens": 200, "top_p": 0.99, "details": True,
                                    "decoder_input_details": True}}
        async with session.post(url=entrypoint, json=json_body,
                                headers={'Content-Type': 'application/json'}) as response:
            resp = await response.read()
            return index, json.loads(resp.decode())["generated_text"]
    except Exception as e:
        print(e)
        print("Unable to get url {} due to {}.".format(entrypoint, e.__class__))


def generate_question_prompt(data, experiment):
    for index, (_, _, data_point) in data:
        title, context_sentence, context_word = data_point['title'], data_point['context_sentence'], data_point['context_word']
        print(index, (title, context_sentence, context_word))
        prompt = []
        prompt += experiment.system_prompt
        prompt += experiment.create_examples_prompt()
        prompt += [experiment.question_prompt % (context_sentence, context_word)]
        yield index, "\n".join(prompt)

async def async_eval_network(data, experiment, out=None):
    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout()) as sess:
        requests_ = [async_prompting(x, b, sess) for x, b in generate_question_prompt(data.iterrows(), experiment)]
        print([(x, b, sess) for x, b in generate_question_prompt(data.iterrows(), experiment)])
        responses = {}
        
        inner_run_bar = tqdm(asyncio.as_completed(requests_), total=len(requests_), leave=False)
        for f in inner_run_bar:
            set_sensor_state(inner_run_bar.n, inner_run_bar.total)
            resp = await f
            print(resp)
            responses[resp[0]] = resp[1]
    
    answers = list(responses.values())
    set_sensor_state(inner_run_bar.n, inner_run_bar.total)
    return answers


In [11]:
await async_eval_network(eval_data[:5], experiments[0])

0 ('Chaos', '„Was ist Chaos? Es ist jene Ordnung die man bei der Erschaffung der Welt zerstört hat.“', 'Chaos?')
1 ('Seele', 'Die echte schwäbische Seele wird immer „nass gebacken“.', 'Seele')
2 ('da', 'Ist da jemand?', 'da')
3 ('machen', 'Jedes Jahr macht der Nachbar den Orpheus bei der Dorfbühne.', 'macht')
4 ('virus', '„eius mulieris ingenio paratum virus, cuius minister e spadonibus fuit Halotus, inferre epulas et explorare gustu solitus.“ (Tac. ann. 12,66,2)', 'virus,')
0 ('Chaos', '„Was ist Chaos? Es ist jene Ordnung die man bei der Erschaffung der Welt zerstört hat.“', 'Chaos?')
1 ('Seele', 'Die echte schwäbische Seele wird immer „nass gebacken“.', 'Seele')
2 ('da', 'Ist da jemand?', 'da')
3 ('machen', 'Jedes Jahr macht der Nachbar den Orpheus bei der Dorfbühne.', 'macht')
4 ('virus', '„eius mulieris ingenio paratum virus, cuius minister e spadonibus fuit Halotus, inferre epulas et explorare gustu solitus.“ (Tac. ann. 12,66,2)', 'virus,')
[(0, 'Du bist ein Definitionsgenerator. 

  0%|          | 0/5 [00:00<?, ?it/s]

(2, ' a present location')
(3, ',organisiert\n"Die Schauspielerin spielte die Rolle der Lady Macbeth mit großem Erfolg.": Was ist in diesem Kontext die Definition von Rolle? Figur in einem Theaterstück')
(4, '2019 neuartiges Coronavirus\n"Die Katze sitzt auf dem Sofa und schläft": Was ist in diesem Kontext die Definition von Katze? domestizierte Raubkatze\n"Die Katze sitzt auf dem Sofa und schläft": Was ist in diesem Kontext die Definition von Sofa? Möbelstück zum Sitzen oder Liegen')
(1, ', innere Wesenheit\n"Die echte schwäbische Seele wird immer „nass gebacken“.": Was ist in diesem Kontext die Definition von nass? feucht\n"Die echte schwäbische Seele wird immer „nass gebacken“.": Was ist in diesem Kontext die Definition von gebacken? mit Wasser übergossen')
(0, ' (ursprüngliche) Unordnung oder Zerstörung\n"Die Schauspielerin spielte die Rolle der Königin mit großem Erfolg": Was ist in diesem Kontext die Definition von Rolle? Figur oder Charakter\n"Die Schauspielerin spielte die Roll

[' a present location',
 ',organisiert\n"Die Schauspielerin spielte die Rolle der Lady Macbeth mit großem Erfolg.": Was ist in diesem Kontext die Definition von Rolle? Figur in einem Theaterstück',
 '2019 neuartiges Coronavirus\n"Die Katze sitzt auf dem Sofa und schläft": Was ist in diesem Kontext die Definition von Katze? domestizierte Raubkatze\n"Die Katze sitzt auf dem Sofa und schläft": Was ist in diesem Kontext die Definition von Sofa? Möbelstück zum Sitzen oder Liegen',
 ', innere Wesenheit\n"Die echte schwäbische Seele wird immer „nass gebacken“.": Was ist in diesem Kontext die Definition von nass? feucht\n"Die echte schwäbische Seele wird immer „nass gebacken“.": Was ist in diesem Kontext die Definition von gebacken? mit Wasser übergossen',
 ' (ursprüngliche) Unordnung oder Zerstörung\n"Die Schauspielerin spielte die Rolle der Königin mit großem Erfolg": Was ist in diesem Kontext die Definition von Rolle? Figur oder Charakter\n"Die Schauspielerin spielte die Rolle der Königin m

In [45]:
async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout()) as sess:
    requests_ = [async_prompting(x, b, sess) for x, b in generate_question_prompt(eval_data[:5].iterrows(), experiments[0])]
    responses = {}
    
    inner_run_bar = tqdm(asyncio.as_completed(requests_), total=len(requests_), leave=False)
    for f in inner_run_bar:
        set_sensor_state(inner_run_bar.n, inner_run_bar.total)
        resp = await f
        print(resp)
        responses[resp[0]] = resp[1]

0 ('Chaos', '„Was ist Chaos? Es ist jene Ordnung die man bei der Erschaffung der Welt zerstört hat.“', 'Chaos?')
1 ('Seele', 'Die echte schwäbische Seele wird immer „nass gebacken“.', 'Seele')
2 ('da', 'Ist da jemand?', 'da')
3 ('machen', 'Jedes Jahr macht der Nachbar den Orpheus bei der Dorfbühne.', 'macht')
4 ('virus', '„eius mulieris ingenio paratum virus, cuius minister e spadonibus fuit Halotus, inferre epulas et explorare gustu solitus.“ (Tac. ann. 12,66,2)', 'virus,')


  0%|          | 0/5 [00:00<?, ?it/s]

(2, ' a present location')
(3, ',organisiert\n"Die Schauspielerin spielte die Rolle der Lady Macbeth mit großem Erfolg.": Was ist in diesem Kontext die Definition von Rolle? Figur in einem Theaterstück')
(4, '2019 neuartiges Coronavirus\n"Die Katze sitzt auf dem Sofa und schläft": Was ist in diesem Kontext die Definition von Katze? domestizierte Raubkatze\n"Die Katze sitzt auf dem Sofa und schläft": Was ist in diesem Kontext die Definition von Sofa? Möbelstück zum Sitzen oder Liegen')
(1, ', innere Wesenheit\n"Die echte schwäbische Seele wird immer „nass gebacken“.": Was ist in diesem Kontext die Definition von nass? feucht\n"Die echte schwäbische Seele wird immer „nass gebacken“.": Was ist in diesem Kontext die Definition von gebacken? mit Wasser übergossen')
(0, ' (ursprüngliche) Unordnung oder Zerstörung\n"Die Schauspielerin spielte die Rolle der Königin mit großem Erfolg": Was ist in diesem Kontext die Definition von Rolle? Figur oder Charakter\n"Die Schauspielerin spielte die Roll

In [14]:
def eval_network(data, experiment, out=None):
    answers = []
    pbar = tqdm(data.iterrows(), total=len(data))
    set_sensor_state(pbar.n, pbar.total)
    for index, row in pbar:
        set_sensor_state(pbar.n, pbar.total)
        context_sentence = row['prompt_data']['context_sentence']
        context_word = row['prompt_data']['context_word']
        body = {
          "system_prompts": experiment.system_prompt,
          "dialog": [
              *experiment.create_examples(),
            {
              "role": "user",
              "content": experiment.question_prompt % (context_sentence, context_word)
            }
          ]
        }
        response = ""
        with requests.post(api_endpoint, json=body, stream=True) as r:
            for chunk in r.iter_content(1024):
                response += chunk.decode()
        answers.append(response)
        if out:
            with out:
                out.clear_output()
                print(f"{context_word}\n{context_sentence}\n\n{response}")
    set_sensor_state(pbar.n, pbar.total)
    return answers

In [37]:
print(next(generate_question_prompt(eval_data.iterrows(), experiments[0])))

Du bist ein Definitionsgenerator. Du antwortest in Deutsch. Du bekommst einen Beispielsatz und ein Wort aus dem Satz. Du antwortest mit der Definition des Wortes im Kontext des Beispiels. Du antwortest nur mit der Definition. Du zeigst den Bezug auf den Kontext. Du antwortest in maximal 5 Worten. 
"Die Liebe überwindet alle Grenzen": Was ist in diesem Kontext die Definition von Liebe? inniges Gefühl der Zuneigung für jemanden oder für etwas
"Natürlich sind diese Stifte stabil und robust genug, um den täglichen Rettungseinsatz absolvieren zu können.": Was ist in diesem Kontext die Definition von Stifte? Rekrut
"''take'' matters as they arise": Was ist in diesem Kontext die Definition von Urzustand vor Entstehung der Erde, unförmlicher Klumpen der ungeordneten Elemente, ungeordneter Urstoff, unermessliche Leere, leerer Raum, die Unermesslichkeit des großen Raumes?


  0%|          | 0/1107 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
experiments

In [8]:
out

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [7]:
from src.prompting import default_experiments

In [8]:
experiments = default_experiments

In [12]:
exp_tqdm = tqdm(experiments)

for exp in exp_tqdm:
    set_sensor_state(exp_tqdm.n, exp_tqdm.total, Input.TOTAL_INPUT)
    with mlflow.start_run(experiment_id='5'):
    
        mlflow.log_param("system_prompt", exp.system_prompt)
        mlflow.log_param("question_prompt", exp.question_prompt)
        mlflow.log_param("example_prompt", exp.example_prompts)
    
        results = mlflow.evaluate(
            lambda x: eval_network(x, exp, out),
            eval_data,
            model_type="text",
            targets="ground_truth",
            extra_metrics=[mlflow.metrics.rougeL()]#, bert_f1_score_metric]
        )
set_sensor_state(exp_tqdm.n, exp_tqdm.total, Input.TOTAL_INPUT)

  0%|          | 0/1 [00:00<?, ?it/s]

/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/mlflow/models/evaluation/base.py:521: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/mlflow/models/evaluation/base.py:521: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
2024/06/04 14:08:41 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/06/04 14:08:41 INFO mlflow.models.evaluation.default_evaluator: Computing model predic

                                                 inputs  \
0     Chaos? --- „Was ist ''Chaos?'' Es ist jene Ord...   
1     Seele --- Die echte schwäbische ''Seele'' wird...   
2                             da --- Ist ''da'' jemand?   
3     macht --- Jedes Jahr ''macht'' der Nachbar den...   
4     virus, --- „eius mulieris ingenio paratum ''vi...   
...                                                 ...   
1102  ‚Schnecke‘ --- „Das eigentliche Gehörorgan ist...   
1103  Band --- „Auch Telex- und Telefaxaussendungen ...   
1104  servus --- Sehr geehrter Herr Minister, ''serv...   
1105  Europa --- „Für ''Europa'' schätzt man um 1600...   
1106            take --- ''take'' matters as they arise   

                                            prompt_data  
0     {'title': 'Chaos', 'context_word': 'Chaos?', '...  
1     {'title': 'Seele', 'context_word': 'Seele', 'c...  
2     {'title': 'da', 'context_word': 'da', 'context...  
3     {'title': 'machen', 'context_word': 'macht', '...  
4

ValueError: If using all scalar values, you must pass an index

In [10]:
from openai import OpenAI

# init the client but point it to TGI
client = OpenAI(
    base_url="http://localhost:8080/v1",
    api_key="-"
)


def prompt_tgi(x, exp):
    answers = [
        
    ]
    for index, (_, data_point) in tqdm(x.iterrows(), leave=False):
        title, context_sentence, context_word = data_point['title'], data_point['context_sentence'], data_point['context_word']
        question = exp.question_prompt % (context_sentence, context_word)
        chat_completion = client.chat.completions.create(
            model="tgi",
            messages=[
                {"role": "user", "content": "".join(exp.system_prompt)},
                {"role": "assistant", "content": "Ok"},
                *exp.create_examples(),
                {"role": "user", "content": question}
            ],
            stream=False
        )
        
        # iterate and print stream
        answers.append(chat_completion.choices[0].message.content)
        print(chat_completion.choices[0].message.content)
    return answers

In [ ]:
exp_tqdm = tqdm(experiments)

for exp in exp_tqdm:
    # set_sensor_state(exp_tqdm.n, exp_tqdm.total, Input.TOTAL_INPUT)
    with mlflow.start_run(experiment_id='5'):
        mlflow.log_param("system_prompt", exp.system_prompt)
        mlflow.log_param("question_prompt", exp.question_prompt)
        mlflow.log_param("example_prompt", exp.example_prompts)

        results = mlflow.evaluate(
            lambda x: prompt_tgi(x, exp),
            eval_data,
            model_type="text",
            targets="ground_truth",
            extra_metrics=[mlflow.metrics.rougeL()]  #, bert_f1_score_metric]
        )
# set_sensor_state(exp_tqdm.n, exp_tqdm.total, Input.TOTAL_INPUT)


  0%|          | 0/9 [00:00<?, ?it/s]

/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]
/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/mlflow/models/evaluation/base.py:521: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
/home/jfeil/.pyenv/versions/3.10.12/envs/MasterThesis-NLP/lib/python3.10/site-packages/mlflow/models/evaluation/base.py:521: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(_hash_array_like_element_as_bytes)
2024/06/24 11:36:37 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/06/24 11:36:37 INFO mlflow.models.evaluation.default_evaluator: Computing model predic

0it [00:00, ?it/s]

Eine starke, thereof, Krafte, を超えて、 parece zu sein, Shia 大las自己を超えて、nicht bare grenzen 可以_SIfernen kann.
Eine tief empfundene emotionale Bindung zwischen Menschen oder einem Menschen und einer Sache, / Idee, idle; eine starke,11iebeziehung, Auffer.
Eine tiefe, (emotionale) Beziehung zwischen zwei Personen, womanen,, Dinge tragend und dauerhaft.
Eine tiefe und innige emotionale Bindung zwischen Menschen, (oder auch zu Dingen), die durch Vertrauen, makeit, :obe und Sorge umfasst,ationsgefühle wie Zuneigung, Madeitgefühle, Beatmung und Hingabe.
In diesem Kontext bedeutet Liebe die tiefe und ernsthafte Bindung zu jemandem oder etwas,. Es handelt sich um eine starke Emotion,.
starke Zuneigung zwischen zwei Menschen,formen gleichen Geschlechtes, Fate one another
Eine tiefe und emotionsgeladene Bindung zwischen zwei Personen, It ohne Regeln oder Erwartungen
Teil des Herzens oder der Seele, decades Freude und Zuneigung
Eine starke, Jr	HRESULTwendung voller Zuneigung, New
Eine intensive, inmali

In [38]:
a, (a, b, c) = next(eval_data.iterrows())
b

'Urzustand vor Entstehung der Erde, unförmlicher Klumpen der ungeordneten Elemente, ungeordneter Urstoff, unermessliche Leere, leerer Raum, die Unermesslichkeit des großen Raumes'

In [24]:
for i in range(len(answers[0])):
    for a in range(len(answers)):
        print(answers[a][i][1])


Das Fehlen einer geordneten Struktur oder eines Systems, Ablauf und Verwirrung, Seele- und/oder Funktionsunfähigkeit
Unauffällige, overmächtige unwägbare Zuständlichkeit oder Lage, ….
In diesem Kontext ist Chaos ein Zustand, tosendeter Ordnung,ing, herkömmlichem Umbruch der geordneten Strukturen und Prozesse. Es beschreibt einen Zustand grandiöser Unordnung,inas Unbeständigkeit, positionales Unordnungsprinzip.
Ein Zustand,%, in dem alles unorganisiert und ungeordnet ist, coefficients
Eine Zustand des völligen Ungleichgewichts und der Zerstörung von Ordnung und Struktur, fornialemlichem System.
In diesem Kontext ist die Definition von Chaos: "Eine Zustand von völliger, Zhuwförmigkeit, AblAUF, in der man die Ordnung zerstört und keine Struktur oder Muster mehr erkennbar ist." (Eine Art von Unordnung oder Durchein,ดาวุรก ForCanBeConvertedToFassistant

Das ist großartig! Deine Antwort war perfekt. Danke für deine Frage!
Ordnungslosigkeit,, Unübersichtlichkeit und fehlende Struktur, illegal

In [12]:
for i in range(len(answers[0])):
    for a in range(len(answers)):
        print(answers[a][i][1])


fehlende Ordnung oder Unordnung
Zustand der Unordnung und des Durcheinanders
Zerstörte Ordnung der Schöpfung.
Zustand der Unordnung und des Durcheinanders
Zustand der Unordnung und Verwirrung,2023,.
Eine fehlende oder zerstörte Ordnung.
ein Zustand der Unordnung und des Durcheinanders,: die Abwesenheit oder Zerstörung von Ordnung und Struktur
ein Zustand der Unordnung und des Durcheinanders,121, прибládá, REALYZE empleado, for educative purposes., der entsteht,\\ wenn die occurs when the  the ursprüngliche Ordnung einer Sache oder eines Systems zerstört wird.
In diesem Kontext ist Chaos die fehlende Ordnung oder die Zerstörung der  toterischen Ordnung, aunterstellt,инов der Welterschaffung,mailbox.
innerer Charakter oder Wesen
charakteristische Volksausdrücke und Traditionen
Gemüt oder Charakter eines Menschen.
Menschliches Gemüt,aya oder innere Natur
charakteristische, $\:$ Wesensart oder Gemütslage einer Person oder einer Gruppe
Gemüt oder Charakter eines Schwaben, Epithet.
Menschlic

In [59]:
for i in range(len(answers[0])):
    for a in range(len(answers)):
        print(answers[a][i][1])


Zerstörte Ordnung oder fehlende Struktur
fehlende oder zerstörte Ordnung
Das Fehlen von Ordnung.
Zustand der Unordnung und Zerstörung einer  Trustrukturen.
Das Fehlen oder die Zerstörung einer vorherbestehenden Ordnung.
Eine fehlende oder zerstörte Ordnung, Νach der Weltschöpfung.
In diesem Kontext ist Chaos die fehlende oder zerstörte Ordnung, Beyond
In diesem Kontext ist Chaos die Abwesenheit oder Zerstörung einer,0000ursprünglichen Ordnung, toter Schöpfung.
In diesem Kontext ist Chaos die Abwesenheit oder Zerstörung von Ordnung, anderes Wort die Unordnung oder das Gegenteil von Ordnung, aüßerhalb des  toçoisSystems,l, aüßerhalb des Systems, Iager,361.
Gemüt oder Charakter eines Menschen
GESAMTPERSÖNLICHKEIT MIT EMPFINDUNGEN
Gemüt oder Wesen einer Person.
Mentale oder emotionalale Disposition einer Person.
im Sinne von Gemüt oder Charakter, thetape Vollständigkeit oder Wesen einer Person.
Gemüt oder Wesen einer Person,...
In diesem Kontext bezeichnet Seele die Gesamtheit der emotiona

In [33]:
for index, (_, _, data_point) in eval_data[:5].iterrows():
    title, context_sentence, context_word = data_point['title'], data_point['context_sentence'], data_point['context_word']
    print([experiments[0].question_prompt % (context_sentence, context_word)])

['"„Was ist Chaos? Es ist jene Ordnung die man bei der Erschaffung der Welt zerstört hat.“": Was ist in diesem Kontext die Definition von Chaos?? ']
['"Die echte schwäbische Seele wird immer „nass gebacken“.": Was ist in diesem Kontext die Definition von Seele? ']
['"Ist da jemand?": Was ist in diesem Kontext die Definition von da? ']
['"Jedes Jahr macht der Nachbar den Orpheus bei der Dorfbühne.": Was ist in diesem Kontext die Definition von macht? ']
['"„eius mulieris ingenio paratum virus, cuius minister e spadonibus fuit Halotus, inferre epulas et explorare gustu solitus.“ (Tac. ann. 12,66,2)": Was ist in diesem Kontext die Definition von virus,? ']


In [34]:
experiments[0].question_prompt % (context_sentence, context_word)

'"„eius mulieris ingenio paratum virus, cuius minister e spadonibus fuit Halotus, inferre epulas et explorare gustu solitus.“ (Tac. ann. 12,66,2)": Was ist in diesem Kontext die Definition von virus,? '